In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
把 osh_structured_incidents.json 寫進 SQLite 資料庫的簡單腳本

功能：
1. 讀取 JSON 檔
2. 建立 SQLite 資料庫 (osh_incidents.db)
3. 建立 incidents 資料表
4. 把所有事故個案寫進資料庫
5. 提供一個簡單關鍵字查詢範例（查描述摘要 description_summary）
"""

import json
import sqlite3
from pathlib import Path

# ==== 依你的環境修改這兩個路徑 =======================================
# JSON 檔路徑（請改成你實際的檔名）
JSON_PATH = Path("/content/osh_structured_incidents (1).json")

# SQLite 資料庫檔案名稱（跑完後同目錄會多一個 .db 檔）
DB_PATH = Path("osh_incidents.db")
# ==================================================================


def load_json(json_path: Path):
    """讀取 JSON 檔，回傳 list[dict]."""
    if not json_path.exists():
        raise FileNotFoundError(f"找不到 JSON 檔案：{json_path}")
    with json_path.open("r", encoding="utf-8") as f:
        data = json.load(f)
    if not isinstance(data, list):
        raise ValueError("預期 JSON 應該是一個 list，每筆是一個 dict。")
    return data


def connect_db(db_path: Path):
    """連線到 SQLite，沒有檔案會自動建立。"""
    conn = sqlite3.connect(db_path)
    return conn


def create_table(conn: sqlite3.Connection):
    """建立 incidents 資料表（如果不存在的話）。"""
    sql = """
    CREATE TABLE IF NOT EXISTS incidents (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        industry TEXT,
        incident TEXT,
        medium_type TEXT,
        description TEXT,
        cause_analysis TEXT,
        preventive_measures TEXT,
        incident_type TEXT,
        incident_type_id TEXT,
        medium_type_general TEXT,
        medium_type_general_id TEXT,
        medium_type_normal TEXT,
        medium_type_normal_id TEXT,
        medium_type_specific TEXT,
        medium_type_specific_id TEXT,
        description_summary TEXT,
        cause_summary TEXT,
        preventive_regulations TEXT
    );
    """
    conn.execute(sql)
    conn.commit()


def insert_incidents(conn: sqlite3.Connection, records: list[dict]):
    """把 JSON 的每一筆資料寫進 incidents 資料表。"""
    sql = """
    INSERT INTO incidents (
        industry,
        incident,
        medium_type,
        description,
        cause_analysis,
        preventive_measures,
        incident_type,
        incident_type_id,
        medium_type_general,
        medium_type_general_id,
        medium_type_normal,
        medium_type_normal_id,
        medium_type_specific,
        medium_type_specific_id,
        description_summary,
        cause_summary,
        preventive_regulations
    ) VALUES (
        :industry,
        :incident,
        :medium_type,
        :description,
        :cause_analysis,
        :preventive_measures,
        :incident_type,
        :incident_type_id,
        :medium_type_general,
        :medium_type_general_id,
        :medium_type_normal,
        :medium_type_normal_id,
        :medium_type_specific,
        :medium_type_specific_id,
        :description_summary,
        :cause_summary,
        :preventive_regulations
    );
    """

    # SQLite 不認識多餘的 key，所以先把每筆 dict 補齊缺的欄位
    normalized_records = []
    keys = [
        "industry",
        "incident",
        "medium_type",
        "description",
        "cause_analysis",
        "preventive_measures",
        "incident_type",
        "incident_type_id",
        "medium_type_general",
        "medium_type_general_id",
        "medium_type_normal",
        "medium_type_normal_id",
        "medium_type_specific",
        "medium_type_specific_id",
        "description_summary",
        "cause_summary",
        "preventive_regulations",
    ]

    for r in records:
        row = {}
        for k in keys:
            row[k] = r.get(k, None)
        normalized_records.append(row)

    conn.executemany(sql, normalized_records)
    conn.commit()


def search_by_keyword(conn: sqlite3.Connection, keyword: str, limit: int = 5):
    """
    簡單示範：用關鍵字搜尋 description_summary，回傳前幾筆。
    未來要給 LLM 用，可以在這裡改成更複雜的查詢。
    """
    sql = """
    SELECT id, industry, incident, description_summary, cause_summary
    FROM incidents
    WHERE description_summary LIKE ?
    LIMIT ?;
    """
    cursor = conn.execute(sql, (f"%{keyword}%", limit))
    return cursor.fetchall()


def main():
    print("=== 第 1 步：讀取 JSON 檔 ===")
    records = load_json(JSON_PATH)
    print(f"讀到 {len(records)} 筆資料。")

    print("=== 第 2 步：建立 / 連線 SQLite 資料庫 ===")
    conn = connect_db(DB_PATH)

    print("=== 第 3 步：建立 incidents 資料表（如果還沒有） ===")
    create_table(conn)

    print("=== 第 4 步：寫入資料 ===")
    # 為避免重複寫入，你可以選擇先清空資料表（可選）
    # conn.execute("DELETE FROM incidents;")
    # conn.commit()

    insert_incidents(conn, records)
    print("資料寫入完成！資料庫檔案：", DB_PATH)

    # 簡單測試查詢
    print("\n=== 第 5 步：簡單查詢示範（關鍵字：『墜落』） ===")
    results = search_by_keyword(conn, "墜落", limit=3)
    for row in results:
        _id, industry, incident, desc_sum, cause_sum = row
        print("-" * 40)
        print(f"ID: { _id }")
        print(f"行業別: {industry}")
        print(f"災害型態: {incident}")
        print(f"描述摘要: {desc_sum}")
        print(f"原因摘要: {cause_sum}")

    conn.close()
    print("\n完成。之後你可以用這個資料庫給 LLM 當查詢後端。")


if __name__ == "__main__":
    main()


=== 第 1 步：讀取 JSON 檔 ===
讀到 760 筆資料。
=== 第 2 步：建立 / 連線 SQLite 資料庫 ===
=== 第 3 步：建立 incidents 資料表（如果還沒有） ===
=== 第 4 步：寫入資料 ===
資料寫入完成！資料庫檔案： osh_incidents.db

=== 第 5 步：簡單查詢示範（關鍵字：『墜落』） ===
----------------------------------------
ID: 762
行業別: 水泥製品製造業
災害型態: 墜落
描述摘要: 罹災者檢修固定式起重機變電盒線路時墜落，送醫不治。
原因摘要: 勞工未架設施工架與工作臺、勞工未使用安全帶與安全帽等防護工具、雇主未禁止勞工搭載堆高機除乘坐席外的位置、雇主未辦理勞工安全衛生教育訓練、雇主未訂定適合之安全衛生工作守則
----------------------------------------
ID: 763
行業別: 未分類其他機械製造修配業
災害型態: 墜落
描述摘要: 罹災者在拆卸起重機手搖鏈條吊車時墜落，送醫不治死亡。
原因摘要: 「勞工未架設施工架與工作臺、勞工未使用安全帶與安全帽等防護工具、勞工未選用適當型式之安全帶及防墜器、雇主未置勞工安全衛生人員、雇主未訂定安全衛生工作守則、雇主未告知再承攬人有關安全衛生規定、雇主未採取共同作業之連繫與調整及巡視措施」
----------------------------------------
ID: 765
行業別: 建築工程業
災害型態: 物體飛落
描述摘要: 8月25日，工地吊籃施工架板料未固定，從30公尺高處墜落，擊中下方工人，送醫不治。
原因摘要: 「勞工未設防止吊掛物通過人員上方及人員進入吊掛物下方之設備或措施、勞工未選用適當吊掛用具、勞工未採取正確吊掛方法及未穩妥固定施工架框架、板料等荷物、雇主未辦理安全衛生教育訓練、雇主未訂定勞工安全衛生工作守則、雇主未設置勞工安全衛生人員及未報備、雇主未訂定自動檢查計畫及實施自動檢查、雇主未將工作環境及危害因素依規定事先告知、雇主未實施各工種吊掛作業之連繫與調整及巡視工作場所」

完成。之後你可以用這個資料庫給 LLM 當查詢後端。
